In [6]:
import os
import pandas as pd
import plotly.graph_objs as go

# Ruta donde están los archivos
ruta_de_archivos = r'C:\Users\rafae\bigData\Dataset de ventas'

# Lista de archivos en la carpeta
archivos = os.listdir(ruta_de_archivos)

# Diccionario para almacenar los DataFrames por mes
dataframes = {}

# Función para extraer y procesar los datos de cada mes
def extraer_datos(datos):
    datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'], format='%d/%m/%Y')
    datos['Hora'] = datos['Fecha de Pedido'].dt.hour
    datos['Dia'] = datos['Fecha de Pedido'].dt.day
    return datos

# Lista de meses en español
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 
         'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

# Lista para almacenar el total de ventas por mes
totales_ventas = []

# Proceso de lectura y análisis de cada archivo CSV
for archivo in archivos:
    # Extraer el nombre del mes desde el nombre del archivo
    mes = archivo.split('_')[1].split('.')[0].lower()  # Ajusta según el nombre de archivo

    if mes in meses:
        ruta_completa = os.path.join(ruta_de_archivos, archivo)
        df = pd.read_csv(ruta_completa)
        
        # Filtrado y procesamiento de los datos
        df_ventas = df.copy()
        df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')
        df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')
        df_ventas['ID de Pedido'] = pd.to_numeric(df_ventas['ID de Pedido'], errors='coerce')

        df_ventas = df_ventas.dropna(subset=['Cantidad Pedida', 'Precio Unitario', 'ID de Pedido'])

        # Convertir los valores a enteros
        df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
        df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)
        
        # Agregar una columna para el total de ventas (Cantidad Pedida * Precio Unitario)
        df_ventas['Total Venta'] = df_ventas['Cantidad Pedida'] * df_ventas['Precio Unitario']

        # Extraer información adicional de la fecha
        df_ventas = extraer_datos(df_ventas)
        
        # Guardar el DataFrame en el diccionario
        dataframes[mes] = df_ventas

        # Calcular el total de ventas del mes
        total_ventas_mes = df_ventas['Total Venta'].sum()
        totales_ventas.append(total_ventas_mes)

# Gráfico de barras del total de ventas por mes
fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale='RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),
    text=totales_ventas
))

fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Total de Ventas'),
)

fig.show()
